In [1]:
%load_ext autoreload
%autoreload 2
import manenv
from manenv.utils.product_utils import *
from manenv.core.world import World
from manenv.core.component import *
from manenv.render_wrapper import RenderWrapper
from manenv.core.product import *
from manenv.core.effector import *

import manenv.effectors as eff
import manenv.components as cmp


pygame-ce 2.4.1 (SDL 2.28.5, Python 3.12.3)


In [ ]:
world = World((3, 3))
s = np.zeros((5, 5), dtype=np.int8)
s[1][2] = 2
s[1][1] = 2
s[2][1] = 1
s[3][1] = 1
product = Product(s)
world.place_component(make_vector(1, 1), cmp.Conveyor(
    np.array([
    [0, 0, 0],
    [1, 0, -1],
    [0, 0, 0]
])))
world.place_component(make_vector(2, 0), cmp.Conveyor(
    np.array([
    [0, -1, 0],
    [0, 0, 1],
    [0, 0, 0] 
])))
world.place_component(make_vector(2, 1), cmp.Outport())

grabber = eff.Grabber(make_vector(0, 2))
buddy = eff.Grabber(make_vector(1, 3))
welder = eff.Welder(make_vector(1, 2))
discard = eff.Acceptor(make_vector(1, 7))

world.place_component(make_vector(1, 2), cmp.Spawner(product))
assembler = cmp.Assembler(make_vector(10, 10), [grabber, buddy, welder, discard])
world.place_component(make_vector(1, 0), assembler)
wrapper = RenderWrapper(world)

: 

In [ ]:
world.reset()
assembler.place_in_inventory(product.copy())
assembler.place_in_inventory(product.copy())
world.update()

grabber.set_action(eff.GrabberActions.GRAB_INVENTORY)
grabber.set_action(eff.GrabberActions.GRAB_INVENTORY)
buddy.set_action(eff.GrabberActions.GRAB_INVENTORY)
world.update()

print(grabber._grabbed_product)

grabber.set_action(eff.GrabberActions.RELEASE)
buddy.set_action(eff.GrabberActions.RELEASE)
world.update()

welder.set_action(eff.WelderActions.WELD_EAST)
world.update()
world.update()

grabber.set_action(eff.GrabberActions.GRAB)
world.update()

grabber.set_action(eff.GrabberActions.MOVE_RIGHT)
world.update()

grabber.set_action(eff.GrabberActions.MOVE_RIGHT)
world.update()

grabber.set_action(eff.GrabberActions.MOVE_RIGHT)
world.update()

grabber.set_action(eff.GrabberActions.MOVE_RIGHT)
world.update()


id: 2082465214
[[2 2]
 [1 0]
 [1 0]]


In [ ]:
wrapper.render(update_mode=True)

TypeError: UIPanel.__init__() missing 1 required positional argument: 'starting_layer_height'